<a href="https://colab.research.google.com/github/qsardor/GoogleColabProjects/blob/main/Tile_Upscaler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title  **Launch the code and wait**
!git clone -q https://huggingface.co/spaces/gokaygokay/Tile-Upscaler app
%cd app
!pip install --quiet -r requirements.txt
!python app.py